In [1]:
import numpy as np
import pandas as pd
import math
import random as rd
from time import perf_counter
import seaborn as sb
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

In [2]:
pd.set_option("display.precision", 10)
df = pd.read_csv("l.txt", index_col = False, sep="\s+", header=None, dtype=np.float64)
df

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,2.0,-1.00362510,0.15284777,-0.547442820,-0.34710145,0.358105450,0.45743140,-0.80935710,-1.76927220,0.021772223,...,0.18384443,-1.121975000,-2.626908800,-1.78834650,0.84040877,2.23012110,-1.51338170,0.21842556,-0.891585890,1.741538600
1,1.0,-0.83159558,-1.56063030,-0.926083850,-0.60384685,-2.252664300,-1.46769520,-1.55183550,-0.71951329,-2.138120200,...,2.21502330,0.975766160,0.816631960,0.40108500,0.12655254,0.17498675,0.28761502,-1.35381760,1.208402100,0.661717260
2,2.0,-0.36107560,0.72168065,-0.263583920,-1.09826050,-0.061373166,1.39068460,1.00134170,-0.48147968,0.348027230,...,1.05438920,0.547787250,0.918944400,-0.91360598,-1.28168850,-0.26522889,0.23069305,1.01305600,0.922631310,-0.612375280
3,2.0,-0.18129062,-0.85974370,-0.908943700,-0.19508274,1.578191100,1.85400610,-0.25136629,0.50622228,-2.192006100,...,0.37050457,0.186187620,2.205772200,0.15883751,-2.13849440,1.74657720,0.32520390,-1.15978060,0.794370210,2.161723700
4,2.0,0.79708788,0.71756099,2.104775500,-0.63907653,0.112702150,0.10302065,-0.65940974,-1.29770860,-0.669483430,...,-0.41767816,-1.375345200,-0.570693420,-2.05980130,-0.15341044,2.05217920,-0.78424387,1.71369660,-0.023950827,-0.090709697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2.0,-1.35628470,-1.30126560,-0.856999220,-0.30793305,-0.060917549,1.13096780,-0.33665089,-0.55236845,0.375775400,...,0.83360880,-0.978734760,1.088950500,-0.17983390,-0.29254703,1.15312610,0.82701521,1.10353990,0.081168392,1.495331900
996,2.0,0.54068848,2.00305640,0.547441260,-1.83495440,1.051617900,2.15462770,-1.07563160,-0.28083650,-0.614868410,...,1.03563380,-1.855254200,-0.040925512,-0.77086456,1.30843820,0.45030879,-0.68578011,1.18860720,0.572931060,-0.625774980
997,2.0,-0.97249132,2.76286380,0.023748010,0.79785204,0.123631410,-0.94783490,-1.09456300,-1.34913500,-0.879244870,...,0.24412129,1.477080000,-1.463541500,0.85205406,-0.73533760,-1.95173540,0.75184235,-0.11929868,-1.077442000,-0.023561353
998,1.0,-1.50931650,0.81474061,0.503906160,-0.39727853,-2.037365800,1.47930040,2.90333140,-0.82706321,-0.566898580,...,-3.12170020,-0.063727814,-0.345365920,0.74504803,-1.20765750,0.71652912,1.73062560,0.22912571,-0.380066210,-1.332631500


# Early stop Forward selection

In [ ]:
def dis(data, feat_opt):
    print("in distance function")
    row = data.shape[0]
    i = 0
    accuracy_counter = 0

    start_time = perf_counter()*1000
    
    while i < row:
    
        idx_predicted_class = 0
        nearest_value = np.inf
    
        j = 0
        while j < row:
            if j != i:
                lpnorm = 0
                for feat_idx in feat_opt: 
                    lpnorm += pow((data.iloc[i,feat_idx] - data.iloc[j,feat_idx]),2)

                if lpnorm < nearest_value:
                    nearest_value = lpnorm
                    idx_predicted_class = j
                    
            j+= 1
        
        if data.iloc[i,0] == data.iloc[idx_predicted_class,0]:
            accuracy_counter += 1
        
        print(f"Finish {i} loop")
        i+=1
    print("calculating time")
    stop_time = perf_counter()*1000
    performance_time = ((stop_time - start_time)/row)
    
    predict_accuracy = (float(accuracy_counter)/row)
    return (performance_time, predict_accuracy)

#forward selection = 1
#backward selection = 0

flag = 1

print("Creating performance data frame")
fearly_stop_performance = pd.DataFrame(columns= ['Time','Average accuracy','Feature set'])
feature_used = []

def display(fu, fa, tm):
    print("in the display function")
    print(f"testing feature {fu} feature accuracy: {round(fa*100,2)}%, performance time {round(tm,2)}ms")
    
i = 1
chance = 0.5

best_accuracy_collector = chance # current list
best_accuracy = chance # testing the current list 

print("Start running the if and else")

if flag:
    
    print("in the forward selection")
    
    feature_collector = []

    print("while loop starting the forward selection")
    
    while i < df.shape[1]:
        
        
        if i not in feature_collector:

            feature_used = feature_collector + [i]
            time_measurement, feature_accuracy = dis(df, feature_used)
            temp_val_holder = [time_measurement, feature_accuracy, feature_used]
            display(feature_used, feature_accuracy, time_measurement)
        
            if feature_accuracy > best_accuracy :
                best_feature = i
                best_accuracy = feature_accuracy

        fearly_stop_performance.loc[len(fearly_stop_performance)] = temp_val_holder

        i+= 1

        if i == df.shape[1]: # end of the feature no more features to loop
            if best_accuracy > best_accuracy_collector:
                feature_collector.append(best_feature)
                best_accuracy_collector = best_accuracy
                i = 1 #restart the loop for the features
else:

    feature_collector = list(range(1,df.shape[1]))

    while i < df.shape[1]:

        if i in feature_collector:

            feature_used = feature_collector.copy()
            feature_used.remove(i)

            time_measurement, feature_accuracy = dis(df, feature_used)
            temp_val_holder = [time_measurement, feature_accuracy, feature_used]
            display(feature_used, feature_accuracy, time_measurement)

            if feature_accuracy > best_accuracy :
                best_feature = i
                best_accuracy = feature_accuracy

        fearly_stop_performance.loc[len(fearly_stop_performance)] = temp_val_holder

        i+= 1

        if i == df.shape[1]: # end of the feature no more features to loop
            if best_accuracy > best_accuracy_collector:
                feature_collector.remove(best_feature)
                best_accuracy_collector = best_accuracy
                i = 1 #restart the loop for the feature}s
print(f"Best Accuracy and Feature: {round(fearly_stop_performance.loc[fearly_stop_performance['Average accuracy'].idxmax()][1],2)*100, fearly_stop_performance.loc[fearly_stop_performance['Average accuracy'].idxmax()][2]}")
print(f"Total time taken: {round((sum(fearly_stop_performance.iloc[:,0])/1000),2)}")

Creating performance data frame
Start running the if and else
in the forward selection
while loop starting the forward selection
in distance function
Started the timer and will begin the looping
Finish 0 loop
Finish 1 loop
Finish 2 loop
Finish 3 loop
Finish 4 loop
Finish 5 loop
Finish 6 loop
Finish 7 loop
Finish 8 loop
Finish 9 loop
Finish 10 loop
Finish 11 loop
Finish 12 loop
Finish 13 loop
Finish 14 loop
Finish 15 loop
Finish 16 loop
Finish 17 loop
Finish 18 loop
Finish 19 loop
Finish 20 loop
Finish 21 loop
Finish 22 loop
Finish 23 loop
Finish 24 loop
Finish 25 loop
Finish 26 loop
Finish 27 loop
Finish 28 loop
Finish 29 loop
Finish 30 loop
Finish 31 loop
Finish 32 loop
Finish 33 loop
Finish 34 loop
Finish 35 loop
Finish 36 loop
Finish 37 loop
Finish 38 loop
Finish 39 loop
Finish 40 loop
Finish 41 loop
Finish 42 loop
Finish 43 loop
Finish 44 loop
Finish 45 loop
Finish 46 loop
Finish 47 loop
Finish 48 loop
Finish 49 loop
Finish 50 loop
Finish 51 loop
Finish 52 loop
Finish 53 loop
Finis

Finish 507 loop
Finish 508 loop
Finish 509 loop
Finish 510 loop
Finish 511 loop
Finish 512 loop
Finish 513 loop
Finish 514 loop
Finish 515 loop
Finish 516 loop
Finish 517 loop
Finish 518 loop
Finish 519 loop
Finish 520 loop
Finish 521 loop
Finish 522 loop
Finish 523 loop
Finish 524 loop
Finish 525 loop
Finish 526 loop
Finish 527 loop
Finish 528 loop
Finish 529 loop
Finish 530 loop
Finish 531 loop
Finish 532 loop
Finish 533 loop
Finish 534 loop
Finish 535 loop
Finish 536 loop
Finish 537 loop
Finish 538 loop
Finish 539 loop
Finish 540 loop
Finish 541 loop
Finish 542 loop
Finish 543 loop
Finish 544 loop
Finish 545 loop
Finish 546 loop
Finish 547 loop
Finish 548 loop
Finish 549 loop
Finish 550 loop
Finish 551 loop
Finish 552 loop
Finish 553 loop
Finish 554 loop
Finish 555 loop
Finish 556 loop
Finish 557 loop
Finish 558 loop
Finish 559 loop
Finish 560 loop
Finish 561 loop
Finish 562 loop
Finish 563 loop
Finish 564 loop
Finish 565 loop
Finish 566 loop
Finish 567 loop
Finish 568 loop
Finish 5

Finish 13 loop
Finish 14 loop
Finish 15 loop
Finish 16 loop
Finish 17 loop
Finish 18 loop
Finish 19 loop
Finish 20 loop
Finish 21 loop
Finish 22 loop
Finish 23 loop
Finish 24 loop
Finish 25 loop
Finish 26 loop
Finish 27 loop
Finish 28 loop
Finish 29 loop
Finish 30 loop
Finish 31 loop
Finish 32 loop
Finish 33 loop
Finish 34 loop
Finish 35 loop
Finish 36 loop
Finish 37 loop
Finish 38 loop
Finish 39 loop
Finish 40 loop
Finish 41 loop
Finish 42 loop
Finish 43 loop
Finish 44 loop
Finish 45 loop
Finish 46 loop
Finish 47 loop
Finish 48 loop
Finish 49 loop
Finish 50 loop
Finish 51 loop
Finish 52 loop
Finish 53 loop
Finish 54 loop
Finish 55 loop
Finish 56 loop
Finish 57 loop
Finish 58 loop
Finish 59 loop
Finish 60 loop
Finish 61 loop
Finish 62 loop
Finish 63 loop
Finish 64 loop
Finish 65 loop
Finish 66 loop
Finish 67 loop
Finish 68 loop
Finish 69 loop
Finish 70 loop
Finish 71 loop
Finish 72 loop
Finish 73 loop
Finish 74 loop
Finish 75 loop
Finish 76 loop
Finish 77 loop
Finish 78 loop
Finish 79 

Finish 533 loop
Finish 534 loop
Finish 535 loop
Finish 536 loop
Finish 537 loop
Finish 538 loop
Finish 539 loop
Finish 540 loop
Finish 541 loop
Finish 542 loop
Finish 543 loop
Finish 544 loop
Finish 545 loop
Finish 546 loop
Finish 547 loop
Finish 548 loop
Finish 549 loop
Finish 550 loop
Finish 551 loop
Finish 552 loop
Finish 553 loop
Finish 554 loop
Finish 555 loop
Finish 556 loop
Finish 557 loop
Finish 558 loop
Finish 559 loop
Finish 560 loop
Finish 561 loop
Finish 562 loop
Finish 563 loop
Finish 564 loop
Finish 565 loop
Finish 566 loop
Finish 567 loop
Finish 568 loop
Finish 569 loop
Finish 570 loop
Finish 571 loop
Finish 572 loop
Finish 573 loop
Finish 574 loop
Finish 575 loop
Finish 576 loop
Finish 577 loop
Finish 578 loop
Finish 579 loop
Finish 580 loop
Finish 581 loop
Finish 582 loop
Finish 583 loop
Finish 584 loop
Finish 585 loop
Finish 586 loop
Finish 587 loop
Finish 588 loop
Finish 589 loop
Finish 590 loop
Finish 591 loop
Finish 592 loop
Finish 593 loop
Finish 594 loop
Finish 5

Finish 38 loop
Finish 39 loop
Finish 40 loop
Finish 41 loop
Finish 42 loop
Finish 43 loop
Finish 44 loop
Finish 45 loop
Finish 46 loop
Finish 47 loop
Finish 48 loop
Finish 49 loop
Finish 50 loop
Finish 51 loop
Finish 52 loop
Finish 53 loop
Finish 54 loop
Finish 55 loop
Finish 56 loop
Finish 57 loop
Finish 58 loop
Finish 59 loop
Finish 60 loop
Finish 61 loop
Finish 62 loop
Finish 63 loop
Finish 64 loop
Finish 65 loop
Finish 66 loop
Finish 67 loop
Finish 68 loop
Finish 69 loop
Finish 70 loop
Finish 71 loop
Finish 72 loop
Finish 73 loop
Finish 74 loop
Finish 75 loop
Finish 76 loop
Finish 77 loop
Finish 78 loop
Finish 79 loop
Finish 80 loop
Finish 81 loop
Finish 82 loop
Finish 83 loop
Finish 84 loop
Finish 85 loop
Finish 86 loop
Finish 87 loop
Finish 88 loop
Finish 89 loop
Finish 90 loop
Finish 91 loop
Finish 92 loop
Finish 93 loop
Finish 94 loop
Finish 95 loop
Finish 96 loop
Finish 97 loop
Finish 98 loop
Finish 99 loop
Finish 100 loop
Finish 101 loop
Finish 102 loop
Finish 103 loop
Finish

Finish 554 loop
Finish 555 loop
Finish 556 loop
Finish 557 loop
Finish 558 loop
Finish 559 loop
Finish 560 loop
Finish 561 loop
Finish 562 loop
Finish 563 loop
Finish 564 loop
Finish 565 loop
Finish 566 loop
Finish 567 loop
Finish 568 loop
Finish 569 loop
Finish 570 loop
Finish 571 loop
Finish 572 loop
Finish 573 loop
Finish 574 loop
Finish 575 loop
Finish 576 loop
Finish 577 loop
Finish 578 loop
Finish 579 loop
Finish 580 loop
Finish 581 loop
Finish 582 loop
Finish 583 loop
Finish 584 loop
Finish 585 loop
Finish 586 loop
Finish 587 loop
Finish 588 loop
Finish 589 loop
Finish 590 loop
Finish 591 loop
Finish 592 loop
Finish 593 loop
Finish 594 loop
Finish 595 loop
Finish 596 loop
Finish 597 loop
Finish 598 loop
Finish 599 loop
Finish 600 loop
Finish 601 loop
Finish 602 loop
Finish 603 loop
Finish 604 loop
Finish 605 loop
Finish 606 loop
Finish 607 loop
Finish 608 loop
Finish 609 loop
Finish 610 loop
Finish 611 loop
Finish 612 loop
Finish 613 loop
Finish 614 loop
Finish 615 loop
Finish 6

Finish 63 loop
Finish 64 loop
Finish 65 loop
Finish 66 loop
Finish 67 loop
Finish 68 loop
Finish 69 loop
Finish 70 loop
Finish 71 loop
Finish 72 loop
Finish 73 loop
Finish 74 loop
Finish 75 loop
Finish 76 loop
Finish 77 loop
Finish 78 loop
Finish 79 loop
Finish 80 loop
Finish 81 loop
Finish 82 loop
Finish 83 loop
Finish 84 loop
Finish 85 loop
Finish 86 loop
Finish 87 loop
Finish 88 loop
Finish 89 loop
Finish 90 loop
Finish 91 loop
Finish 92 loop
Finish 93 loop
Finish 94 loop
Finish 95 loop
Finish 96 loop
Finish 97 loop
Finish 98 loop
Finish 99 loop
Finish 100 loop
Finish 101 loop
Finish 102 loop
Finish 103 loop
Finish 104 loop
Finish 105 loop
Finish 106 loop
Finish 107 loop
Finish 108 loop
Finish 109 loop
Finish 110 loop
Finish 111 loop
Finish 112 loop
Finish 113 loop
Finish 114 loop
Finish 115 loop
Finish 116 loop
Finish 117 loop
Finish 118 loop
Finish 119 loop
Finish 120 loop
Finish 121 loop
Finish 122 loop
Finish 123 loop
Finish 124 loop
Finish 125 loop
Finish 126 loop
Finish 127 lo

# Backward stop backward selection

In [ ]:
def dis(data, feat_opt):
    
    row = data.shape[0]
    i = 0
    accuracy_counter = 0

    start_time = perf_counter()*1000
    
    while i < row:
        
        idx_predicted_class = 0
        nearest_value = np.inf
    
        j = 0
        while j < row:
            if j != i:
                lpnorm = 0
                for feat_idx in feat_opt: 
                    lpnorm += pow((data.iloc[i,feat_idx] - data.iloc[j,feat_idx]),2)

                #print(f"i: {i}, j: {j}, lpnorm: {lpnorm}")
                if lpnorm < nearest_value:
                    nearest_value = lpnorm
                    idx_predicted_class = j
                    
            j+= 1
        
        if data.iloc[i,0] == data.iloc[idx_predicted_class,0]:
            accuracy_counter += 1
            
        i+=1
    
    stop_time = perf_counter()*1000
    performance_time = ((stop_time - start_time)/row)
    
    predict_accuracy = (float(accuracy_counter)/row)
    return (performance_time, predict_accuracy)

#forward selection = 1
#backward selection = 0

flag = 0

bearly_stop_performance = pd.DataFrame(columns= ['Time','Average accuracy','Feature set'])
feature_used = []

def display(fu, fa, tm):
    print(f"testing feature {fu} feature accuracy: {round(fa*100,2)}%, performance time {round(tm,2)}ms")
    
i = 1
chance = 0.5

best_accuracy_collector = chance # current list
best_accuracy = chance # testing the current list 

if flag:
    feature_collector = []

    while i < df.shape[1]:

        if i not in feature_collector:

            feature_used = feature_collector + [i]
            time_measurement, feature_accuracy = dis(df, feature_used)
            temp_val_holder = [time_measurement, feature_accuracy, feature_used]
            display(feature_used, feature_accuracy, time_measurement)

            if feature_accuracy > best_accuracy :
                best_feature = i
                best_accuracy = feature_accuracy

        bearly_stop_performance.loc[len(bearly_stop_performance)] = temp_val_holder

        i+= 1

        if i == df.shape[1]: # end of the feature no more features to loop
            if best_accuracy > best_accuracy_collector:
                feature_collector.append(best_feature)
                best_accuracy_collector = best_accuracy
                i = 1 #restart the loop for the features
else:

    feature_collector = list(range(1,df.shape[1]))

    while i < df.shape[1]:

        if i in feature_collector:

            feature_used = feature_collector.copy()
            feature_used.remove(i)

            time_measurement, feature_accuracy = dis(df, feature_used)
            temp_val_holder = [time_measurement, feature_accuracy, feature_used]
            display(feature_used, feature_accuracy, time_measurement)

            if feature_accuracy > best_accuracy :
                best_feature = i
                best_accuracy = feature_accuracy

        bearly_stop_performance.loc[len(bearly_stop_performance)] = temp_val_holder

        i+= 1

        if i == df.shape[1]: # end of the feature no more features to loop
            if best_accuracy > best_accuracy_collector:
                feature_collector.remove(best_feature)
                best_accuracy_collector = best_accuracy
                i = 1 #restart the loop for the feature}s
print(f"Best Accuracy and Feature: {round(bearly_stop_performance.loc[bearly_stop_performance['Average accuracy'].idxmax()][1],2)*100, bearly_stop_performance.loc[bearly_stop_performance['Average accuracy'].idxmax()][2]}")
print(f"Total time taken: {round((sum(bearly_stop_performance.iloc[:,0])/1000),2)}")

testing feature [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40] feature accuracy: 69.1%, performance time 2089.78ms
testing feature [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40] feature accuracy: 70.0%, performance time 2079.18ms
testing feature [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40] feature accuracy: 68.2%, performance time 2076.58ms
testing feature [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40] feature accuracy: 71.7%, performance time 2074.47ms
testing feature [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,

# Without early stop forward selection

In [ ]:
def dis(data, feat_opt):
    
    row = data.shape[0]
    i = 0
    accuracy_counter = 0

    start_time = perf_counter()*1000
    
    while i < row:
        
        idx_predicted_class = 0
        nearest_value = np.inf
    
        j = 0
        while j < row:
            if j != i:
                lpnorm = 0
                for feat_idx in feat_opt: 
                    lpnorm += pow((data.iloc[i,feat_idx] - data.iloc[j,feat_idx]),2)

                #print(f"i: {i}, j: {j}, lpnorm: {lpnorm}")
                if lpnorm < nearest_value:
                    nearest_value = lpnorm
                    idx_predicted_class = j
                    
            j+= 1
        
        if data.iloc[i,0] == data.iloc[idx_predicted_class,0]:
            accuracy_counter += 1
            
        i+=1
    
    stop_time = perf_counter()*1000
    performance_time = ((stop_time - start_time)/row)
    
    predict_accuracy = (float(accuracy_counter)/row)
    return (performance_time, predict_accuracy)

#forward selection = 1
#backward selection = 0

flag = 1

cc = pd.DataFrame(columns= ['Time','Average accuracy','Feature set'])
feature_used = []

def display(fu, fa, tm):
    print(f"testing feature {fu} feature accuracy: {round(fa*100,2)}%, performance time {round(tm,2)}ms")
    
i = 1

best_accuracy_collector = -1 # current list
best_accuracy = -1 # testing the current list 

if flag:
    feature_collector = []

    while i < df.shape[1]:

        if i not in feature_collector:

            feature_used = feature_collector + [i]
            time_measurement, feature_accuracy = dis(df, feature_used)
            temp_val_holder = [time_measurement, feature_accuracy, feature_used]
            display(feature_used, feature_accuracy, time_measurement)

            if feature_accuracy > best_accuracy :
                best_feature = i
                best_accuracy = feature_accuracy

        wesfs_performance.loc[len(wesfs_performance)] = temp_val_holder

        i+= 1

        if i == df.shape[1]: # end of the feature no more features to loop
            
            if best_accuracy > best_accuracy_collector:
                
                feature_collector.append(best_feature)
                best_accuracy_collector = best_accuracy
                i = 1 #restart the loop for the features
                best_accuracy = -1
                
            else: #accuracy is not going to get better here
                feature_collector.append(best_feature)
                if len(feature_collector) < df.shape[1]:
                    best_accuracy = -1
                    i = 1
                
                
else:

    feature_collector = list(range(1,df.shape[1]))

    while i < df.shape[1]:

        if i in feature_collector:

            feature_used = feature_collector.copy()
            feature_used.remove(i)

            time_measurement, feature_accuracy = dis(df, feature_used)
            temp_val_holder = [time_measurement, feature_accuracy, feature_used]
            display(feature_used, feature_accuracy, time_measurement)

            if feature_accuracy > best_accuracy :
                best_feature = i
                best_accuracy = feature_accuracy

        wesfs_performance.loc[len(wesfs_performance)] = temp_val_holder

        i+= 1

        if i == df.shape[1]: # end of the feature no more features to loop
            
            if best_accuracy > best_accuracy_collector:
                feature_collector.remove(best_feature)
                best_accuracy_collector = best_accuracy
                i = 1 #restart the loop for the feature}s
                best_accuracy = -1 
                
            else:
                
                feature_collector.remove(best_feature)
                
                if len(feature_collector) > 0: #accuracy is not going to get better here
                    best_accuracy = -1
                    i = 1
                
                
                
                
print(f"Best Accuracy and Feature: {round(wesfs_performance.loc[wesfs_performance['Average accuracy'].idxmax()][1],2)*100, wesfs_performance.loc[wesfs_performance['Average accuracy'].idxmax()][2]}")
print(f"Total time taken: {round((sum(wesfs_performance.iloc[:,0])/1000),2)}s")

# Without early stop backward selection

In [ ]:
def dis(data, feat_opt):
    
    row = data.shape[0]
    i = 0
    accuracy_counter = 0

    start_time = perf_counter()*1000
    
    while i < row:
        
        idx_predicted_class = 0
        nearest_value = np.inf
    
        j = 0
        while j < row:
            if j != i:
                lpnorm = 0
                for feat_idx in feat_opt: 
                    lpnorm += pow((data.iloc[i,feat_idx] - data.iloc[j,feat_idx]),2)

                #print(f"i: {i}, j: {j}, lpnorm: {lpnorm}")
                if lpnorm < nearest_value:
                    nearest_value = lpnorm
                    idx_predicted_class = j
                    
            j+= 1
        
        if data.iloc[i,0] == data.iloc[idx_predicted_class,0]:
            accuracy_counter += 1
            
        i+=1
    
    stop_time = perf_counter()*1000
    performance_time = ((stop_time - start_time)/row)
    
    predict_accuracy = (float(accuracy_counter)/row)
    return (performance_time, predict_accuracy)

#forward selection = 1
#backward selection = 0

flag = 0

wesbs_performance = pd.DataFrame(columns= ['Time','Average accuracy','Feature set'])
feature_used = []

def display(fu, fa, tm):
    print(f"testing feature {fu} feature accuracy: {round(fa*100,2)}%, performance time {round(tm,2)}ms")
    
i = 1

best_accuracy_collector = -1 # current list
best_accuracy = -1 # testing the current list 

if flag:
    feature_collector = []

    while i < df.shape[1]:

        if i not in feature_collector:

            feature_used = feature_collector + [i]
            time_measurement, feature_accuracy = dis(df, feature_used)
            temp_val_holder = [time_measurement, feature_accuracy, feature_used]
            display(feature_used, feature_accuracy, time_measurement)

            if feature_accuracy > best_accuracy :
                best_feature = i
                best_accuracy = feature_accuracy

        wesbs_performance.loc[len(wesbs_performance)] = temp_val_holder

        i+= 1

        if i == df.shape[1]: # end of the feature no more features to loop
            
            if best_accuracy > best_accuracy_collector:
                
                feature_collector.append(best_feature)
                best_accuracy_collector = best_accuracy
                i = 1 #restart the loop for the features
                best_accuracy = -1
                
            else: #accuracy is not going to get better here
                feature_collector.append(best_feature)
                if len(feature_collector) < df.shape[1]:
                    best_accuracy = -1
                    i = 1
                
                
else:

    feature_collector = list(range(1,df.shape[1]))

    while i < df.shape[1]:

        if i in feature_collector:

            feature_used = feature_collector.copy()
            feature_used.remove(i)

            time_measurement, feature_accuracy = dis(df, feature_used)
            temp_val_holder = [time_measurement, feature_accuracy, feature_used]
            display(feature_used, feature_accuracy, time_measurement)

            if feature_accuracy > best_accuracy :
                best_feature = i
                best_accuracy = feature_accuracy

        wesbs_performance.loc[len(wesbs_performance)] = temp_val_holder

        i+= 1

        if i == df.shape[1]: # end of the feature no more features to loop
            
            if best_accuracy > best_accuracy_collector:
                feature_collector.remove(best_feature)
                best_accuracy_collector = best_accuracy
                i = 1 #restart the loop for the feature}s
                best_accuracy = -1 
                
            else:
                
                feature_collector.remove(best_feature)
                
                if len(feature_collector) > 0: #accuracy is not going to get better here
                    best_accuracy = -1
                    i = 1
                
                
                
                
print(f"Best Accuracy and Feature: {round(wesbs_performance.loc[wesbs_performance['Average accuracy'].idxmax()][1],2)*100, wesbs_performance.loc[wesbs_performance['Average accuracy'].idxmax()][2]}")
print(f"Total time taken: {round((sum(wesbs_performance.iloc[:,0])/1000),2)}s")

In [ ]:
fearly_stop_performance = fearly_stop_performance.astype({'Feature set':'string'})

fearly_stop_performance.dtypes

In [ ]:
fearly_stop_performance = fearly_stop_performance.astype({'Feature set':'string'})
fearly_stop_performance.dtypes
figure(figsize =(26,8), dpi=720)
sb.lineplot(data=fearly_stop_performance, x="Feature set", y="Time").set(title='Time Used for Each Features\nForward Selection with Early Stopping')
sb.despine()
plt.xticks(rotation=90)
plt.xlabel('Feature set')
plt.ylabel('Time')
plt.show()

In [ ]:
fearly_stop_performance.dtypes
figure(figsize =(26,8), dpi=720)
sb.lineplot(data=fearly_stop_performance, x="Feature set", y="Average accuracy").set(title='Time Used for Each Feature set\n-Forward Selection with Early Stopping')
plt.xticks(rotation=90)
plt.xlabel('Feature set')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
wesfs_performance = wesfs_performance.astype({'Feature set':'string'})
figure(figsize=(26,8), dpi=720)
sb.lineplot(data=wesfs_performance, x="Feature set", y="Time").set(title='Time Used for Each Feature set\nfackward Selection Without Early Stopping')
plt.xticks(rotation=90)
plt.xlabel('Feature Set')
plt.ylabel('Time')
plt.show()


In [ ]:
figure(figsize=(26,8), dpi=720)
sb.lineplot(data=wesfs_performance, x="Feature set", y="Average accuracy").set(title='Accuracy\nForward Selection Without Early Stopping')
plt.xticks(rotation=90)
plt.xlabel('Feature Set')
plt.ylabel('Accuracy')
plt.show()


# Backwards Selection Plots

In [ ]:
bearly_stop_performance = bearly_stop_performance.astype({'Feature set':'string'})
bearly_stop_performance.dtypes
figure(figsize =(26,8), dpi=720)
sb.lineplot(data=bearly_stop_performance, x="Feature set", y="Time").set(title='Time Used for Each Feature set\nBackward Selection with Early Stopping')
plt.xticks(rotation=90)
plt.xlabel('Feature set')
plt.ylabel('Average accuracy')
plt.show()

In [ ]:
figure(figsize =(26,8), dpi=720)
sb.lineplot(data=bearly_stop_performance, x="Feature set", y="Average accuracy").set(title='Accuracy\nBackward Selection with Early Stopping')
plt.xticks(rotation=90)
plt.xlabel('Feature set')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
wesbs_performance = wesbs_performance.astype({'Feature set':'string'})
figure(figsize =(26,8), dpi=720)
sb.lineplot(data=wesbs_performance, x="Feature set", y="Time").set(title='Time Used for Each Feature set\nBackward Selection with Early Stopping')
plt.xticks(rotation=90)
plt.xlabel('Feature set')
plt.ylabel('Time(ms)')
plt.show()

In [ ]:
figure(figsize =(26,8), dpi=720)
sb.lineplot(data=wesbs_performance, x="Feature set", y="Average accuracy").set(title='Time Used for Each Feature set\nBackward Selection with Early Stopping')
plt.xticks(rotation=90)
plt.xlabel('Feature set')
plt.ylabel('Average accuracy')
plt.show()